
# Data Preparation


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.cluster import SpectralClustering
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pickle
import numpy as np
import time

from sklearn.preprocessing import OneHotEncoder


import statsmodels.api as sm
from EKAN_functions import *


# SETTING UP PANDAS VISUALIZATION
pd.set_option('display.max_columns', None)

#LOADING DATASET

data = pd.read_csv('your_data.csv')
data=data.drop(columns=['Unnamed: 0'],axis=1)
#data=data.drop(columns=['Unnamed: 0.1'],axis=1)
#data=data.drop(columns=['center'],axis=1)
data.head()


In [ ]:
# CREATE DUMMY FOR DX CLASSES
from sklearn.preprocessing import LabelBinarizer
bin=LabelBinarizer()
Dx_cols=bin.fit_transform(data['Dx'])

Dx_cols=pd.DataFrame(Dx_cols)
Dx_cols.columns=['HC','CHR','ROP','ROD']
Dx_cols.head()

# ADD ROP AND ROP TO DATA TO PERFORM THE CORRECTION 
data=pd.concat([data,Dx_cols[['ROP','ROD']]],axis=1)

## DATASET PREPARATION 

In [ ]:
# In order to maintain data integrity and make it balanced we choose to perform a preliminary classification on 
# HC vs ROP+ROD and a second one with a different model on ROP vs ROD thiss will prevent the negative effec
# of unbalanced dataset on model training, that we can see on the confusion matrix (multiclass script).
# therefore the following steps are necessary



## DATASET PREPARATION FOR THE FIRST MODEL
# Dropping CHR patients

data = data.drop(data[data['Dx'] == 1].index) 
# Binarize the labels (healthy vs non healthy)
data['Dx'] = np.where(data['Dx'] > 0, 1, 0)


## DATASET PREPARATION FOR THE SECOND MODEL
'''
# REMOVE HC AND CHR
data = data.drop(data[data['Dx'] == 0].index) 
data = data.drop(data[data['Dx'] == 1].index) 

# Binarize the labels (ROP vs ROD)
data['Dx'] = np.where(data['Dx'] > 2, 1, 0)


#data
'''

In [ ]:
data.shape

# DEFINITION OF THE PIPELINE

In [ ]:


# Import the BioCovariatesRegression class from module
path = r'the path to your folder'
import sys 
sys.path.insert(0,path)

from sklearn.pipeline import Pipeline
# Use confounder_correction_classes_1 to model the Dx effect on the correction
# Use confounder_correction_classes if you don't want to model the Dx effet
from new_confounder_correction_classes_1 import BiocovariatesRegression , StandardScalerDict , ComBatHarmonization

# INITIALIZATION

# choose the number of features you want to correct
vol_feat=np.arange(0,393) # 9 substances + confounders features and 73 sMRI features + 319 fMRI + Dx
binary_feat=np.arange(6,9) # smoker, non smoker, occasional smoker

# Features that need to be corrected in the LR model with covariates 
#feat_of_interest={'volumes': {'id': vol_feat, 'categorical': ['SEX','Dx'], 'continuous': ['AGE']}}
feat_of_interest={'volumes': {'id': vol_feat, 'categorical': ['SEX','ROP','ROD'], 'continuous': ['AGE']}} #take into account ROP and ROD

## INITIALIZE CORRECTION FUNCTIONS ##

combat_harm=ComBatHarmonization(cv_method=None,ref_batch=None,regression_fit=1,feat_detail=feat_of_interest,feat_of_no_interest=None)
biocov_regre=BiocovariatesRegression(cv_method=None,feat_detail=feat_of_interest,ref_batch=None,output_dict=False)#,binary_feat=binary_feat)
scaler_data_fun=StandardScalerDict(std_data=True,std_cov=False,output_dict=True,ref_batch=None)

# CREATE PROCESSING PIPELINE #

steps_biocov=[('Apply combat Harmonization',combat_harm),('Standardize only data', scaler_data_fun),('Confounder Regression', biocov_regre),('Standardize', StandardScaler())]
pipeline_biocov_reg= Pipeline(steps_biocov)



# DEFINITION OF CV DATASET BUILDER

In [ ]:
from sklearn.model_selection import train_test_split

y = data['Dx']#.astype(int)
X = data.drop(['Dx'], axis = 1)

#SPLIT DATA INTO TRAIN AND TEST SET
X_train, X_test, y_train, y_test = train_test_split(X, y,  #X_scaled
                                                    test_size =0.30, #by default is 75%-25%
                                                    #shuffle is set True by default,
                                                    stratify=y,
                                                    random_state = 123) #fix random seed for replicability

print(X_train.shape)



n_splits=5
n_repeats=2
number_of_folds=n_splits*n_repeats
cv = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=2)#random_state=1) set 2 for ROP vs ROD

# CREATION OF CV_DATASET FOR THE OUTER LOOP (TRAIN TEST)
#[X_train_gs,y_train_gs,cv_gs]=CV_dataset_builder(X_train,y_train,cv,pipeline_biocov_reg)

# CREATION OF CV_DATASET FOR THE OUTER LOOP (THE ONE WE USED)
[X_train_gs,y_train_gs,cv_gs]=CV_dataset_builder(X,y,cv,pipeline_biocov_reg)

X_train_gs.to_csv('your_folds_corrected_x.csv')
y_train_gs.to_csv('your_folds_corrected_y.csv')


# Save the array to a file
with open("your_folds_pointers", "wb") as f:
    pickle.dump(cv_gs, f)

In [ ]:
# CORRECT ALL THE DATASET TO TRAIN THE FINAL MODEL


binary_var=['smoker','occasional_smoker','non_smoker','DRINKER'] # for mediator is correct because binary variables are not used
confounder_var=['SEX','AGE','ROP','ROD','batch']
## ALLOCATING DATA ON DIFFERENT VARIABLES AND PREPARING FOR THE FIT ##
# Copy the database on other independent variables
X_train_v=X.copy()
#X_train_v=pd.concat([X_train_v,y_train], axis=1)


#X_test_v=X_test.copy()
#X_test_v=pd.concat([X_test_v,y_test], axis=1)

# SPLIT BINARY AND CONTINUOUS VARIABLES

X_b_train_v=X_train_v[binary_var+confounder_var].copy()
#X_b_test_v=X_test_v[binary_var+confounder_var].copy()
X_train_v=X_train_v.drop(binary_var,axis=1)
#X_test_v=X_test_v.drop(binary_var,axis=1)
  
# Save confounders in other db & Remove them from the main data, to avoid overflow error
    
# Save variables
X_c_train_v=X_train_v[confounder_var].copy()
#X_c_test_v=X_test_v[confounder_var].copy()

# drop them from the main dataset
X_train_v=X_train_v.drop(confounder_var,axis=1).copy()
#X_test_v=X_test_v.drop(confounder_var,axis=1).copy()

##### CONTINUOUS #####------------------------------------------------------

# Build the dictionary structure to ease data correction
X_train_v_dict={'data': X_train_v, 'covariates': X_c_train_v}
#X_test_v_dict={'data': X_test_v, 'covariates': X_c_test_v}


## PERFORMING THE CORRECTION ##

# Train the models that apply the correction on data
pipeline_biocov_reg.fit(X_train_v_dict) # Fit pipeline on the training set

# Apply the correction estimated previously
X_train_v_corr=pipeline_biocov_reg.transform(X_train_v_dict) # Apply fitted pipeline on train
#X_test_v_corr=pipeline_biocov_reg.transform(X_test_v_dict) # Apply fitted pipeline on test


## SAVE CORRECTED DATA ##

# Overwrite the corrected data on the uncorrected one
for i in range(0,len(X_train_v_corr)):
    X_train_v.iloc[i]=X_train_v_corr[i].copy() #apply correction on test sample to the train set


#for i in range(0,len(X_test_v_corr)):
    #X_test_v.iloc[i]=X_test_v_corr[i].copy() #apply correction on test sample to the test set

#---------------------------------------------------------------------

##### BINARY #####----------------------------------------------------
covars=['Dx','SEX','AGE','batch']
binary_var=['smoker','occasional_smoker','non_smoker','DRINKER']
'''
for feat in binary_var:
       
    #--This part is common for both train and test beacuse is the training of the regression--

    est = sm.Logit(X_b_train_v[feat],X_b_train_v[confounder_var]) 
    estimates=est.fit()
    estimates.params[2]=0 # remove effect of Dx


    # save raw smoker var
    if feat == 'smoker':
        y_s_train=X_b_train_v[feat].copy()
        y_s_test=X_b_test_v[feat].copy()

    #---
    for i in range(0,len(X_b_train_v[feat])):
        # Substitue the logit value to the old one for train
        X_b_train_v[feat].iloc[i]=max(0,X_b_train_v[feat].iloc[i]-estimates.predict(X_b_train_v[confounder_var].iloc[i])[0])
        
    for i in range(0,len(X_b_test_v[feat])):
        # do the same for test
        X_b_test_v[feat].iloc[i]=max(0,X_b_test_v[feat].iloc[i]-estimates.predict(X_b_test_v[confounder_var].iloc[i])[0])
'''




X_b_train_v=X_b_train_v.drop(confounder_var,axis=1).copy()
#X_b_test_v=X_b_test_v.drop(confounder_var,axis=1).copy()

#----------------------------------------------------------------------------

##### CONCAT BINARY AND CONTINUOUS #####-------------------------------------

X_train_v=pd.concat([X_b_train_v,X_train_v],axis=1)
#X_test_v=pd.concat([X_b_test_v,X_test_v],axis=1)


#SAVE DATA FOR TRAIN
y_train=y

X_train_v.to_csv('your_dataset_corrected_X.csv')
y_train.to_csv('your_dataset_corrected_y.csv')



